In [390]:
import pandas as pd
import zipfile
import kaggle
from matplotlib import pyplot as plt 
import seaborn as sns

import numpy as np 
import plotly.express as px 
import seaborn as sns
from plotly.offline import iplot , plot 
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objects as go


<h1>Data Gathering </>

In [391]:
#Download dataset from kaggle using Kaggle API 
!kaggle datasets download -d hummaamqaasim/jobs-in-data

jobs-in-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [392]:
#Extract the file from the downloaded Zip file
zipfile_name = 'jobs-in-data.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [393]:
#Read the csv file 
df = pd.read_csv('jobs_in_data.csv')

In [394]:
df.head()

,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81800,United States,Senior,Full-time,In-person,United States,M
3,2023,Data Scientist,Data Science and Research,USD,212000,212000,United States,Senior,Full-time,In-person,United States,M
4,2023,Data Scientist,Data Science and Research,USD,93300,93300,United States,Senior,Full-time,In-person,United States,M


<h1> Data cleaning </h1>

In [395]:
#Let's look at data types  
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9355 entries, 0 to 9354
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           9355 non-null   int64 
 1   job_title           9355 non-null   object
 2   job_category        9355 non-null   object
 3   salary_currency     9355 non-null   object
 4   salary              9355 non-null   int64 
 5   salary_in_usd       9355 non-null   int64 
 6   employee_residence  9355 non-null   object
 7   experience_level    9355 non-null   object
 8   employment_type     9355 non-null   object
 9   work_setting        9355 non-null   object
 10  company_location    9355 non-null   object
 11  company_size        9355 non-null   object
dtypes: int64(3), object(9)
memory usage: 877.2+ KB


In [396]:
#Let's look at the number of NA values
df.isna().sum()

work_year             0
job_title             0
job_category          0
salary_currency       0
salary                0
salary_in_usd         0
employee_residence    0
experience_level      0
employment_type       0
work_setting          0
company_location      0
company_size          0
dtype: int64

In [397]:
#Let's look at the descriptive statistics 
df.describe()

,work_year,salary,salary_in_usd
count,9355.000000,9355.000000,9355.000000
mean,2022.760449,149927.981293,150299.495564
std,0.519470,63608.835387,63177.372024
min,2020.000000,14000.000000,15000.000000
25%,2023.000000,105200.000000,105700.000000
50%,2023.000000,143860.000000,143000.000000
75%,2023.000000,187000.000000,186723.000000
max,2023.000000,450000.000000,450000.000000


In [398]:
#We only want to look at the salaries for Data analysts, Data scientist and Data engineers 

df['job_title'] = df['job_title'].str.lower()
#job_title_list = ['data analyst', 'data scientist', 'data engineer']

#df = df[df['job_title'].isin(job_title_list)]

In [399]:
pd.DataFrame({'Count':df.shape[0],
              'Null':df.isnull().sum(),
              'Null %':df.isnull().mean() * 100,
              'Cardinality':df.nunique()
})

,Count,Null,Null %,Cardinality
work_year,9355,0,0.0,4
job_title,9355,0,0.0,125
job_category,9355,0,0.0,10
salary_currency,9355,0,0.0,11
salary,9355,0,0.0,1507
salary_in_usd,9355,0,0.0,1786
employee_residence,9355,0,0.0,83
experience_level,9355,0,0.0,4
employment_type,9355,0,0.0,4
work_setting,9355,0,0.0,3


In [400]:
# Before beginning with the visualizations, lets find and remove the outliers from the data as we want to focus on the average salaries per criteria.

# To find outliers we will use the Inter Quartile Range method. 

# Calculate quartiles
Q1 = df['salary_in_usd'].quantile(0.25)
Q3 = df['salary_in_usd'].quantile(0.75)

# Calculate IQR (Interquartile Range)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = (df['salary_in_usd'] < lower_bound) | (df['salary_in_usd'] > upper_bound)

# Remove outliers
df = df[~outliers]

# Print the result
print("DataFrame without outliers:\n", df)


DataFrame without outliers:
       work_year                 job_title                    job_category  \
0          2023      data devops engineer                Data Engineering   
1          2023            data architect  Data Architecture and Modeling   
2          2023            data architect  Data Architecture and Modeling   
3          2023            data scientist       Data Science and Research   
4          2023            data scientist       Data Science and Research   
...         ...                       ...                             ...   
9349       2021             data engineer                Data Engineering   
9350       2021           data specialist    Data Management and Strategy   
9352       2021  principal data scientist       Data Science and Research   
9353       2020            data scientist       Data Science and Research   
9354       2020     business data analyst                   Data Analysis   

     salary_currency  salary  salary_in_usd em

<h1>Visualisations</h1>

<h3> Trend in salaries (USD) </h3>

In [401]:
df_salary_trend = df.groupby('work_year')['salary_in_usd'].mean()

In [402]:
fig = iplot(px.line(df_salary_trend,
             markers=True,
             labels={'work_year':'Year','value':'Salary In USD'},
             title='Trend in salaries (USD)',
             template='plotly_dark',
             line_shape="linear",
             color_discrete_sequence=['#cc2114']
             ))


<p style="text-align:center; font-weight:bold;">Insights</p>

Growth in average salary between 2020 and 2023 appears to be very high from approximately USD 95,000 to 150,000. <br>
Growth in average salary between 2020 and 2021 was small and linear from approximately USD 95,000 to 110,000. <br>
Growth in average salary between 2021 and 2022 was comparatively larger from approximately USD 110,000 to USD 135,000.<br>
Growth in average salary between 2022 and 2023 saw a similar pattern to the hike in between 2021 and 2022 from approximately $135,000 to USD 150,000. 

<h3>Number of employees per job title</h3>

In [403]:
df_employees_job_title = df.groupby('job_category')['salary_in_usd'].mean()

In [404]:
iplot(px.line(df_employees_job_title,
             markers=True,
             labels={'job_category':'Job Category','value':'Salary In USD'},
             title='Average salary by job category',
             template='plotly_dark',
             line_shape="linear",
             color_discrete_sequence=['#cc2114']
             ))

<p style="text-align:center; font-weight:bold;">Insights</p>
On an average jobs in the category Machine learning and AI and Data science and research pay the highest salary upwards of USD 160,000.<br>
Average salaries in job category of Cloud and Database, Data Architecture and Modelling and Data Engineering are notably high. 

<h3>Salaries by Job title</h3>

In [405]:
df_job_title_salary = df.groupby('job_title')['salary_in_usd'].sum()

In [406]:
iplot(px.bar(df_job_title_salary.sort_values(ascending=False)[:10],
             orientation='h',
             labels={'value':'Salary in USD','job_title':'Job Title'},
             title='Job Title with Salaries USD',
             template='plotly_dark',
             color=df_job_title_salary.index[:10],
             text_auto=True
))

Insights:
These values are based on the sum of salaries of all employees separated by job titles.<br> 
Sum of salaries for Data engineers and data scientists are notably high. <br>
A reason for this could be that number of employees for the above mentioned job titles are comparatively higher than any other job titles. <br>
Data engineers make the most amount of salary closely followed by data scientists. <br>
Top four job titles exhibit strong earning potential in the job market. <br>

Lets look at Average salaries by job title instead 

<H3>Average salaries by job title</H3>

In [407]:
df_average_job_title_salary = df.groupby('job_title')['salary_in_usd'].mean()

In [408]:
iplot(px.bar(df_average_job_title_salary.sort_values(ascending=False)[:10],
             orientation='h',
             labels={'value':'Salary in USD','job_title':'Job Title'},
             title='Average salaries in USD by job title',
             template='plotly_dark',
             color=df_average_job_title_salary.index[:10],
             text_auto=True
))

<p style="text-align:center; font-weight:bold;">Insights</p>

Managing Director data science makes the highest average salary. Average salaries for cloud and AWS data architects are also notably high. <br>
The top 10 Job titles show a strong earning potential.

Let's now look average salaries by company location for the top four job titles:
1. Data engineer
2. Data Scientist
3. Machine learning engineer
4. Data Analyst 

<h2>Average salary for Data Engineer by country</h2>

In [409]:
df_de_salary_country = df[df['job_title'] == 'data engineer']
df_de_salary_country = df_de_salary_country.groupby('company_location')['salary_in_usd'].mean()

In [410]:
iplot(px.bar(df_de_salary_country.sort_values(ascending=False)[:10],
             orientation='h',
             labels={'value':'Salary in USD','company_location':'Location'},
             title='Average Salary in USD for Data engineer by location (Top 10)',
             template='plotly_dark',
             color=df_de_salary_country.sort_values(ascending=False)[:10],
             text_auto=True
))

<p style="text-align:center; font-weight:bold;">Insights</p>
Data engineers working in Puerto rico, United states and canada on an average get paid above USD 140,000.  

<h2>Average salary for Data science by country</h2>

In [411]:
df_ds_salary_country = df[df['job_title'] == 'data scientist']
df_ds_salary_country = df_ds_salary_country.groupby('company_location')['salary_in_usd'].mean()

In [412]:
iplot(px.bar(df_ds_salary_country.sort_values(ascending=False)[:10],
             orientation='h',
             labels={'value':'Salary in USD','company_location':'Location'},
             title='Average Salary in USD for Data Scientist by location (Top 10)',
             template='plotly_dark',
             color=df_ds_salary_country.sort_values(ascending=False)[:10],
             text_auto=True
))

<p style="text-align:center; font-weight:bold;">Insights</p>
Data scientists working in Canada, United states and Switzerland on an average get paid above USD 120,000.  

<h2>Average salary for Machine learning engineer by country</h2>

In [413]:
df_ml_salary_country = df[df['job_title'] == 'machine learning engineer']
df_ml_salary_country = df_ml_salary_country.groupby('company_location')['salary_in_usd'].mean()

In [414]:
iplot(px.bar(df_ml_salary_country.sort_values(ascending=False)[:10],
             orientation='h',
             labels={'value':'Salary in USD','company_location':'Location'},
             title='Average Salary in USD for Machine Learning Engineer by location (Top 10)',
             template='plotly_dark',
             color=df_ml_salary_country.sort_values(ascending=False)[:10],
             text_auto=True
))

<p style="text-align:center; font-weight:bold;">Insights</p>
Machine learning engineers working in Ukraine, Australia,  United states and canada on an average get paid above USD 170,000.  

<h2>Average salary for Data analytics by country</h2>

In [415]:
df_da_salary_country = df[df['job_title'] == 'data analyst']
df_da_salary_country = df_da_salary_country.groupby('company_location')['salary_in_usd'].mean()

In [416]:
iplot(px.bar(df_da_salary_country.sort_values(ascending=False)[:10],
             orientation='h',
             labels={'value':'Salary in USD','company_location':'Location'},
             title='Average Salaries USD by location (Top 10)',
             template='plotly_dark',
             color=df_da_salary_country.sort_values(ascending=False)[:10],
             text_auto=True
))

<p style="text-align:center; font-weight:bold;">Insights</p>
Data analysts in Australia, United states and Canada on an average get paid above USD 100,000.  <br>
A data analyst wishing to get paid above USD 100,000 should work in the above mentioned countries. 

<h3>Average salary by company size</h3>

In [417]:
df_salary_company_size = df.groupby('company_size')['salary_in_usd'].mean()

In [418]:
iplot(px.bar(df_salary_company_size,
             labels={'value':'Salary in USD','company_size':'Company Size'},
             title='Average Salaries in USD by company size',
             template='plotly_dark',
             color=df_salary_company_size.index,
             text_auto=True))

<p style="text-align:center; font-weight:bold;">Insights</p>
On an average a medium sized company pays more to their employees closely followed by Large size. <br>
Small size company pays the least. 

<h3>Average Salary by experience</h3>

In [419]:
df_salary_experience = df.groupby('experience_level')['salary_in_usd'].mean()

In [420]:
iplot(px.bar(df_salary_experience,
             labels={'value':'Salary in USD','experience_level':'Experience Level'},
             title='Average Salaries USD by employee experience size',
             template='plotly_dark',
             color=df_salary_experience.index,
             text_auto=True
             ))

<p style="text-align:center; font-weight:bold;">Insights</p>
The insight here is as expected:
On an average an executive level employee has highest salary followed by senior, mid -level and entry-level employee. <br>

<H3>Average Salary by employment type</H3>

In [421]:
df_salary_employment_type = df.groupby('employment_type')['salary_in_usd'].mean()

In [422]:
iplot(px.bar(df_salary_employment_type,
             labels={'value':'Salary in USD','employment_type':'Employment Type'},
             title='Average Salaries USD by employment type',
             template='plotly_dark',
             color=df_salary_employment_type.index,
             text_auto=True
             ))

<p style="text-align:center; font-weight:bold;">Insights</p>

On an average a full time employee makes more than an employee on a different employment type. 